In [1]:
import mediapipe as mp
import cv2
import pandas as pd
import pickle

import warnings
warnings.filterwarnings('ignore')

# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

objc[39879]: Class CaptureDelegate is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x118cc8860) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x28ceee480). One of the two will be used. Which one is undefined.
objc[39879]: Class CVWindow is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x107a98a68) and /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/cv2/cv2.abi3.so (0x28ceee4d0). One of the two will be used. Which one is undefined.
objc[39879]: Class CVView is implemented in both /Users/fuixlabsdev1/Programming/PP/graduation-thesis/env/lib/python3.8/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x107a98a90) and /Users/fuixlabsdev1/Programming/PP/graduation-th

### 1. Train Model

#### 1.1. Describe data and split dataset

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import confusion_matrix

In [19]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)


def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

In [4]:
df = describe_dataset("./train.csv")

df.tail(3)

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'lef

,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
28517,H,0.735630,0.543294,0.007467,0.999246,0.695831,0.417349,0.155194,0.995723,0.720067,...,0.086010,0.966131,0.226601,0.598075,0.219305,0.470830,0.220079,0.614120,0.026265,0.934942
28518,H,0.775572,0.517579,0.012821,0.999378,0.704168,0.404210,0.162908,0.995909,0.730823,...,0.070911,0.967070,0.238810,0.610591,0.198591,0.496140,0.228907,0.625559,0.018591,0.938905
28519,H,0.790600,0.498958,0.007789,0.999467,0.710651,0.394019,0.164441,0.996123,0.736771,...,0.085872,0.967943,0.238197,0.609329,0.233198,0.510583,0.227823,0.626068,0.036127,0.940917


In [5]:
# Extract features and class
X = df.drop("label", axis=1)
y = df["label"]

In [7]:
sc = StandardScaler()
X = pd.DataFrame(sc.fit_transform(X))

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
y_test.head(3)

1469    C
292     C
1568    C
Name: label, dtype: object

#### 1.2. Train model using Scikit-Learn and train set evaluation

In [35]:
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score

In [51]:
algorithms =[("LR", LogisticRegression()),
         ("SVC", SVC(probability=True)),
         ('KNN',KNeighborsClassifier()),
         ("DTC", DecisionTreeClassifier()),
         ("SGDC", CalibratedClassifierCV(SGDClassifier())),
         ("NB", GaussianNB()),
         ('RF', RandomForestClassifier()),]

models = {}
final_results = []

for name, model in algorithms:
    trained_model = model.fit(X_train, y_train)
    models[name] = trained_model

    # Evaluate model
    model_results = model.predict(X_test)

    p_score = precision_score(y_test, model_results, average=None, labels=["C", "L", "H"])
    a_score = accuracy_score(y_test, model_results)
    r_score = recall_score(y_test, model_results, average=None, labels=["C", "L", "H"])
    f1_score_result = f1_score(y_test, model_results, average=None, labels=["C", "L", "H"])
    cm = confusion_matrix(y_test, model_results, labels=["C", "L", "H"])
    final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm))


In [52]:
# Sort results by F1 score
final_results.sort(key=lambda k: sum(k[4]), reverse=True)

pd.DataFrame(final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,KNN,"[0.999, 1.0, 1.0]",0.999825,"[1.0, 0.999, 1.0]","[1.0, 1.0, 1.0]","[[1915, 0, 0], [1, 1944, 0], [0, 0, 1844]]"
1,LR,"[0.999, 0.999, 1.0]",0.999649,"[0.999, 0.999, 1.0]","[0.999, 0.999, 1.0]","[[1914, 1, 0], [1, 1944, 0], [0, 0, 1844]]"
2,SVC,"[0.998, 0.999, 1.0]",0.999299,"[0.999, 0.998, 1.0]","[0.999, 0.999, 1.0]","[[1914, 1, 0], [3, 1942, 0], [0, 0, 1844]]"
3,RF,"[0.998, 1.0, 1.0]",0.999299,"[1.0, 0.998, 0.999]","[0.999, 0.999, 1.0]","[[1915, 0, 0], [3, 1942, 0], [1, 0, 1843]]"
4,SGDC,"[0.998, 0.999, 0.998]",0.998597,"[0.997, 0.999, 0.999]","[0.998, 0.999, 0.999]","[[1910, 2, 3], [2, 1943, 0], [1, 0, 1843]]"
5,DTC,"[0.995, 0.998, 1.0]",0.997721,"[0.999, 0.995, 0.999]","[0.997, 0.997, 0.999]","[[1913, 2, 0], [9, 1936, 0], [0, 2, 1842]]"
6,NB,"[0.816, 0.941, 0.931]",0.892532,"[0.883, 0.847, 0.951]","[0.848, 0.892, 0.941]","[[1690, 103, 122], [290, 1648, 7], [91, 0, 1753]]"


#### 1.3. Test set evaluation

In [23]:
test_df = describe_dataset("./test.csv")
test_df = test_df.sample(frac=1).reset_index(drop=True)

test_x = test_df.drop("label", axis=1)
test_y = test_df["label"]

test_x = pd.DataFrame(sc.transform(test_x))

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v', 'left_knee_x', 'left_knee_y', 'left_knee_z', 'left_knee_v', 'right_knee_x', 'right_knee_y', 'right_knee_z', 'right_knee_v', 'left_ankle_x', 'left_ankle_y', 'left_ankle_z', 'left_ankle_v', 'right_ankle_x', 'right_ankle_y', 'right_ankle_z', 'right_ankle_v', 'left_heel_x', 'left_heel_y', 'left_heel_z', 'left_heel_v', 'right_heel_x', 'right_heel_y', 'right_heel_z', 'right_heel_v', 'left_foot_index_x', 'lef

In [50]:
testset_final_results = []

for name, model in models.items():
    # Evaluate model
    model_results = model.predict(test_x)

    p_score = precision_score(test_y, model_results, average=None, labels=["C", "L", "H"])
    a_score = accuracy_score(test_y, model_results)
    r_score = recall_score(test_y, model_results, average=None, labels=["C", "L", "H"])
    f1_score_result = f1_score(test_y, model_results, average=None, labels=["C", "L", "H"])
    cm = confusion_matrix(test_y, model_results, labels=["C", "L", "H"])
    testset_final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm ))


testset_final_results.sort(key=lambda k: sum(k[4]), reverse=True)
pd.DataFrame(testset_final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,LR,"[0.987, 1.0, 1.0]",0.995775,"[1.0, 0.991, 0.996]","[0.994, 0.996, 0.998]","[[234, 0, 0], [2, 233, 0], [1, 0, 240]]"
1,SGDC,"[0.979, 0.996, 1.0]",0.991549,"[1.0, 0.991, 0.983]","[0.989, 0.994, 0.992]","[[234, 0, 0], [2, 233, 0], [3, 1, 237]]"
2,SVC,"[0.971, 1.0, 1.0]",0.990141,"[1.0, 0.979, 0.992]","[0.985, 0.989, 0.996]","[[234, 0, 0], [5, 230, 0], [2, 0, 239]]"
3,KNN,"[0.87, 1.0, 1.0]",0.950704,"[1.0, 0.86, 0.992]","[0.93, 0.924, 0.996]","[[234, 0, 0], [33, 202, 0], [2, 0, 239]]"
4,RF,"[0.745, 1.0, 1.0]",0.887324,"[1.0, 0.66, 1.0]","[0.854, 0.795, 1.0]","[[234, 0, 0], [80, 155, 0], [0, 0, 241]]"
5,NB,"[0.892, 0.945, 0.737]",0.842254,"[0.632, 0.949, 0.942]","[0.74, 0.947, 0.827]","[[148, 13, 73], [4, 223, 8], [14, 0, 227]]"
6,DTC,"[0.682, 0.643, 1.0]",0.770423,"[0.577, 0.766, 0.963]","[0.625, 0.699, 0.981]","[[135, 99, 0], [55, 180, 0], [8, 1, 232]]"


#### 1.4. Dumped model and input scaler using pickle

According to the evaluations, there are multiple good models at the moment, therefore, the best models are LR and Ridge.

In [40]:
with open("./model/all_sklearn.pkl", "wb") as f:
    pickle.dump(models, f)

In [41]:
with open("./model/LR_model.pkl", "wb") as f:
    pickle.dump(models["LR"], f)

In [42]:
with open("./model/SGDC_model.pkl", "wb") as f:
    pickle.dump(models["SGDC"], f)

In [21]:
# Dump input scaler
with open("./model/input_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)